In [1]:
spark

In [2]:
#import statements
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import *
from datetime import datetime

In [3]:
spark = SparkSession.builder.appName('Final_project').getOrCreate()
sc = spark.sparkContext

In [76]:
path_comments = "/user/rroongseang/bigdata/comments/"
path_users = "/user/rroongseang/bigdata/users/"

In [77]:
df = spark.read.json(path_comments+"politics_json*")
users = spark.read.csv(path_users+"RA.2019-09.csv", inferSchema=True, header=True)

In [78]:
from pyspark.sql.types import IntegerType
#df = df.withColumn("created_utc", df["created_utc"].cast(IntegerType()))
df = df.withColumn("ups", df["ups"].cast(IntegerType()))
df = df.withColumn("downs", df["downs"].cast(IntegerType()))
df = df.withColumn("gilded", df["gilded"].cast(IntegerType()))
df = df.withColumn("score", df["score"].cast(IntegerType()))
df = df.withColumn("retrieved_on", df["retrieved_on"].cast(IntegerType()))
df = df.withColumn("controversiality", df["controversiality"].cast(IntegerType()))

from pyspark.sql.functions import unix_timestamp
df = df.withColumn('created_utc',from_utc_timestamp(df["created_utc"],'CST'))

In [80]:
df.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: integer (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: integer (nullable = true)
 |-- gilded: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: integer (nullable = true)
 |-- score: integer (nullable = true)
 |-- score_hidden: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: integer (nullable = true)



In [44]:
users = users.select("id", col("name").alias("username"), col("created_utc").alias("acct_creation"),"updated_on","comment_karma","link_karma")

In [45]:
users.printSchema()

root
 |-- id: long (nullable = true)
 |-- username: string (nullable = true)
 |-- acct_creation: integer (nullable = true)
 |-- updated_on: integer (nullable = true)
 |-- comment_karma: integer (nullable = true)
 |-- link_karma: integer (nullable = true)



In [9]:
df_combined = df.join(users, df.author == users.name, 'left')

In [10]:
df_combined.select(df.author,df.body,df.ups,df.downs,df.created_utc,users.created_utc,users.name).show(5)

+-----------+--------------------+---+-----+-----------+-----------+----+
|     author|                body|ups|downs|created_utc|created_utc|name|
+-----------+--------------------+---+-----+-----------+-----------+----+
|---Eagle---|Is that why he lo...|  1| null| 1456898256|       null|null|
|---Eagle---|Most victims of I...| 14| null| 1449931915|       null|null|
|---Eagle---|&gt; We could hav...|  9| null| 1445179831|       null|null|
|---Eagle---|The guy spends 15...|  1| null| 1457429478|       null|null|
|---Eagle---|He's a rabid zion...| -1| null| 1450858290|       null|null|
+-----------+--------------------+---+-----+-----------+-----------+----+
only showing top 5 rows



In [11]:
df_combined.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: string (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: string (nullable = true)
 |-- gilded: string (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: string (nullable = true)
 |-- score: string (nullable = true)
 |-- score_hidden: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- created_utc: integer (nullable = true)
 |-- updated_on: integer (nullable = true)
 |-- comment_ka

In [18]:
utc = 1450858290
datetime.fromtimestamp(utc)

datetime.datetime(2015, 12, 23, 2, 11, 30)

In [31]:
df_combined.select("parent_id").dropna().count()

3209623

In [82]:
users.count()

71631739